In [ ]:
import os
from dotenv import load_dotenv
from huggingface_hub import login
from datasets import load_dataset, concatenate_datasets
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
# import numpy as np
# import random

load_dotenv(override=True)

/workspaces/js-complexity-model/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [ ]:
from utils.get_complexity import get_complexity_label, get_complexity_score

In [3]:
hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

Token has not been saved to git credential helper.
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.


In [11]:
dataset_dict = load_dataset("semeru/code-text-javascript")

In [12]:
print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url', 'partition'],
        num_rows: 58025
    })
    validation: Dataset({
        features: ['repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url', 'partition'],
        num_rows: 3885
    })
    test: Dataset({
        features: ['repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url', 'partition'],
        num_rows: 3291
    })
})


In [18]:
def transform_data(datapoint):
    score = get_complexity_score(datapoint.get("code", ""))
    return {
        "complexity": score,
        "label": get_complexity_label(score)
    }

processed_ds = dataset_dict.map(transform_data, num_proc=4)
processed_ds = processed_ds.select_columns(["repo", "code", "complexity", "label"])

Map (num_proc=4):   0%|          | 0/58025 [00:00<?, ? examples/s]

Map (num_proc=4): 100%|██████████| 3291/3291 [00:00<00:00, 3421.38 examples/s]


In [20]:
print(processed_ds["train"][0])

{'repo': 'ciena-blueplanet/bunsen-core', 'code': 'function (state, action) {\n    return _.defaults({\n      isValidating: action.isValidating,\n      lastAction: IS_VALIDATING\n    }, state)\n  }', 'complexity': 1.3, 'label': 'Simple'}


In [21]:
processed_ds.push_to_hub("aslam-naseer/js-function-complexity")

Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00,  4.00ba/s]
Processing Files (1 / 1): 100%|██████████| 15.5MB / 15.5MB,  505kB/s  
New Data Upload: 100%|██████████| 15.5MB / 15.5MB,  505kB/s  
Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 47.87ba/s]
Processing Files (1 / 1): 100%|██████████| 1.02MB / 1.02MB,  364kB/s  
New Data Upload: 100%|██████████| 1.02MB / 1.02MB,  364kB/s  
Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 67.34ba/s]
Processing Files (1 / 1): 100%|██████████|  980kB /  980kB,  350kB/s  
New Data Upload: 100%|██████████|  980kB /  980kB,  350kB/s  
Uploading the dataset shards: 100%|██████████| 1/1 [00:03<00:00,  3.55s/ shards]


CommitInfo(commit_url='https://huggingface.co/datasets/aslam-naseer/js-function-complexity/commit/6d8b3386d0b20a6ae4a0ddb8db5b179a3158e05c', commit_message='Upload dataset', commit_description='', oid='6d8b3386d0b20a6ae4a0ddb8db5b179a3158e05c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/aslam-naseer/js-function-complexity', endpoint='https://huggingface.co', repo_type='dataset', repo_id='aslam-naseer/js-function-complexity'), pr_revision=None, pr_num=None)

In [14]:
# TRAIN_SIZE = 5000
# VAL_SIZE = 500
# TEST_SIZE = 500

In [15]:
# total_split = TRAIN_SIZE + VAL_SIZE + TEST_SIZE 

# ds = dataset.select_columns(["original_string"])

# main_split = ds.train_test_split(test_size=total_split, seed=42, shuffle=True)
# pool_6k = main_split["test"]

# eval_size = VAL_SIZE + TEST_SIZE
# train_eval_split = pool_6k.train_test_split(test_size=eval_size, seed=42)

# train_ds = train_eval_split["train"] 
# temp_1k  = train_eval_split["test"]

# final_split = temp_1k.train_test_split(test_size=TEST_SIZE, seed=42)
# val_ds  = final_split["train"] 
# test_ds = final_split["test"]

# print(f"Final Sizes - Train: {len(train_ds)}, Test: {len(test_ds)}, Val: {len(val_ds)}")

In [16]:
# def process_item(example):
#     # This runs your existing function on the string
#     processed_code = preprocess_js_code(example)
#     return {
#         "processed_string": processed_code,
#         "length": len(processed_code)
#     }

# # Apply to your splits
# train_ds = train_ds.map(process_item)
# val_ds   = val_ds.map(process_item)
# test_ds  = test_ds.map(process_item)


In [17]:
# # Plot the distribution of lengths
# full_set = concatenate_datasets([train_ds, test_ds, val_ds])
# lengths = full_set["length"]

# plt.figure(figsize=(15, 6))
# plt.title(f"Text length: Avg {sum(lengths)/len(lengths):,.1f} and highest {max(lengths):,}\n")
# plt.xlabel('Length (characters)')
# plt.ylabel('Count')
# plt.hist(lengths, rwidth=0.7, color="skyblue", bins=range(0, 2500, 50))
# plt.show()